In [1]:
#todo 
# 모든 값들이 32byte가 되도록 padding을 해줘야함

In [2]:
import struct

def countbytes(target):
    if isinstance(target, str):
        b = bytes(target,'utf-8')
    elif isinstance(target, int):
        b = target.to_bytes((target.bit_length() + 7) // 8, 'big')
    else:
        b = target
    print(f"original: {target}")
    print(f"byte value : {b}")
    print(f"size : {len(b)} bytes  ")


In [3]:
import collections
import random
import time

EllipticCurve = collections.namedtuple('EllipticCurve', 'name p a b g n h')

#curve for NIST p-256
curve = EllipticCurve(
    'p256',
    # Field characteristic.
    p=0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff,
    # Curve coefficients.
    a=0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc,
    b=0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b,
    # Base point.
    g=(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296,
        0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5),
    # Subgroup order.
    n=0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551,
    # Subgroup cofactor.
    h=1,
)

# Modular arithmetic ##########################################################
def inverse_mod(k, p):
    """Returns the inverse of k modulo p.
    This function returns the only integer x such that (x * k) % p == 1.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - inverse_mod(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p


# Functions that work on curve points #########################################

def is_on_curve(point):
    """Returns True if the given point lies on the elliptic curve."""
    if point is None:
        # None represents the point at infinity.
        return True

    x, y = point

    return (y * y - x * x * x - curve.a * x - curve.b) % curve.p == 0


def point_add(point1, point2):
    """Returns the result of point1 + point2 according to the group law."""
    assert is_on_curve(point1)
    assert is_on_curve(point2)

    if point1 is None:
        # 0 + point2 = point2
        return point2
    if point2 is None:
        # point1 + 0 = point1
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        # point1 + (-point1) = 0
        return None

    if x1 == x2:
        # This is the case point1 == point2.
        m = (3 * x1 * x1 + curve.a) * inverse_mod(2 * y1, curve.p)
    else:
        # This is the case point1 != point2.
        m = (y1 - y2) * inverse_mod(x1 - x2, curve.p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p, -y3 % curve.p)

    assert is_on_curve(result)

    return result


def scalar_mult(k, point):
    """Returns k * point computed using the double and point_add algorithm."""
    assert is_on_curve(point)

    if k % curve.n == 0 or point is None:
        return None

    if k < 0:
        # k 가 음수일 경우 절대값으로 양수로 만듦 (현재 설정에선 음수가 나올 일은 없음)
        # k * point = -k * (-point)
        return scalar_mult(-k, point_neg(point))

    result = None
    addend = point

    while k:
        if k & 1:
            # Add.
            result = point_add(result, addend)

        # Double.
        addend = point_add(addend, addend)

        k >>= 1

    assert is_on_curve(result)

    return result

# Keypair generation and ECDSA ################################################
def make_keypair():
    """Generates a random private-public key pair."""
    private_key = random.randrange(1, curve.n)
    public_key = scalar_mult(private_key, curve.g)

    return private_key, public_key


# Challenge process

In [4]:
from Crypto import Random
from Crypto.Cipher import AES
#Generate Nonce 
import binascii
import os 
import struct 
import datetime 

BLOCK_SIZE = 16
def pad(s):
    s = s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE).encode()
    # print(f"padding : {countbytes(s)}")
    return s

def unpad(encrypted_string):
    return encrypted_string[0:-(encrypted_string[-1])]


class AESCipher(object):
    def __init__(self, key):
        self.key = key.encode()

    def encrypt(self, raw):
        raw = pad(raw)
        iv = Random.new().read(AES.block_size)
        # print(f"iv : {countbytes(iv)}")
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        # print(f"cipher len : {len(cipher.encrypt(raw))}")
        return iv + cipher.encrypt(raw)
    
    def decrypt(self, enc):
        iv = enc[:BLOCK_SIZE] 
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(enc[BLOCK_SIZE:]))

def AES_key_padding(z):
    if len(z) > 32 : 
        z = z[:32]
    else:
        for i in range(32-len(z)):
            z = z+"0"
    return z 

# Generate Keys

In [5]:
#input verification code (6 digits) 
registerCode = 123456
username = "AUTOCRYPT"

In [6]:
# 5.1 Gen. ECDH Key pair for Phone
# create key pair for mobile and car 
phoneSecretKey, phonePublicKey = make_keypair() # phone side 
carSecretKey, carPublicKey = make_keypair() # car side 

In [7]:
# 5.2 Send Phone Pub key
## exchange public keys
    #phone sends public key to car
    #car sends public key to phone

### Car --> Phone Challenge

In [8]:
# 5.6.  Gen confirmation Key (Register Code)
# calculate shared secret
phoneSharedKey= scalar_mult(phoneSecretKey, carPublicKey)[0] # phone side 
phoneVerificationKey= scalar_mult(phoneSecretKey, carPublicKey)[1] # phone side 

carSharedKey = scalar_mult(carSecretKey, phonePublicKey)[0] # car side`
carVerificationKey = scalar_mult(carSecretKey, phonePublicKey)[1] # car side`

#calculate confirmation key  value
phoneConfirmationKey = (phoneVerificationKey + registerCode) % curve.n # phone side
carConfirmationKey = (carVerificationKey + registerCode) % curve.n # car side

### Car --> Phone Challenge

In [9]:
# 6.1 Gen. challenge = nonce_c(4Byte), Timestamp_c (4Byte)
# generate challenge
nonce_c = int(binascii.hexlify(os.urandom(4)),16).to_bytes(4,"big")# 4byte random int
ts = struct.pack(">i",int(datetime.datetime.timestamp(datetime.datetime.now())))
challenge = nonce_c+ts

In [10]:
#6.3 Shared key, confirmation key 순서로 2중 암호화

AES_key1 = AES_key_padding(str(carSharedKey)) # 1st encryption with shared key 
AES_key2 = AES_key_padding(str(carConfirmationKey)) # 2nd encryption with confirmation key 

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 shared key 를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 confirmation key를 Key로 암호화 인스턴스2 생성 

# todo 혼다측 요구사항 (2중암호화)을 위해 register code로 한번 더 암호화 해야됨 
# shared_password로 1차 암호화 한 후 
# spake를 통해 만들어진 Ke로 2차 암호화 

plain_challenge_from_car = challenge
# print("plain    :",plain_challenge_from_car)
countbytes(plain_challenge_from_car)

encrypted_challenge_from_car1 = Car_cipher1.encrypt(plain_challenge_from_car)
# print("encrypted1:",encrypted_challenge_from_car1)
countbytes(encrypted_challenge_from_car1)

encrypted_challenge_from_car2 = Car_cipher2.encrypt(encrypted_challenge_from_car1)
# print("encrypted2:",encrypted_challenge_from_car2)
countbytes(encrypted_challenge_from_car2)


original: b'\x86\x02u\x8dct\x8a\xe8'
byte value : b'\x86\x02u\x8dct\x8a\xe8'
size : 8 bytes  
original: b'\x92x6\xd2\x0c6\x0eI\xb67<|\xe4\xca\x9e\x07\x00B\xc5j\xc8\x98-\x8d\x8f~\xe9\xac\xe7\xd7\x96\xa7'
byte value : b'\x92x6\xd2\x0c6\x0eI\xb67<|\xe4\xca\x9e\x07\x00B\xc5j\xc8\x98-\x8d\x8f~\xe9\xac\xe7\xd7\x96\xa7'
size : 32 bytes  
original: b'E\x88s!\x9f\x16\xa0\xf2\xd1\x00t\xb4\xd0\x16u$\xe2Q\xabm+E27\x8f3)o^\x8a\xa9\x14\x0b6\x81\xeb\xa5\x00\xc6\x9b\xedT\xf6\xb7\xcc\xb0\x7f\xfc\xef\x08\x80\x83\xeee9\x11\xb3\xb9b\xce\x19\x11~H'
byte value : b'E\x88s!\x9f\x16\xa0\xf2\xd1\x00t\xb4\xd0\x16u$\xe2Q\xabm+E27\x8f3)o^\x8a\xa9\x14\x0b6\x81\xeb\xa5\x00\xc6\x9b\xedT\xf6\xb7\xcc\xb0\x7f\xfc\xef\x08\x80\x83\xeee9\x11\xb3\xb9b\xce\x19\x11~H'
size : 64 bytes  


### car --> phone Challenge solve
### Phone --> Car Challenge

In [11]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(str(phoneSharedKey)) # 1st encryption with shared key 
AES_key2 = AES_key_padding(str(phoneConfirmationKey)) # 2nd encryption with confirmation key 

Phone_cipher1 = AESCipher(key=AES_key1)  # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Phone_cipher2 = AESCipher(key=AES_key2)  #  2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

# 2번 복호화 
decrypted_challenge_from_car1 = Phone_cipher2.decrypt(encrypted_challenge_from_car2)
decrypted_challenge_from_car = Phone_cipher1.decrypt(decrypted_challenge_from_car1)
countbytes(decrypted_challenge_from_car)

padded_username = int(username.encode().hex(),16).to_bytes(64,"big")
countbytes(padded_username)

plain_response_from_phone = decrypted_challenge_from_car+padded_username
countbytes(plain_response_from_phone)

# 2중암호화 
encrypted_response_from_phone1 = Phone_cipher1.encrypt(plain_response_from_phone)
countbytes(encrypted_response_from_phone1)
encrypted_response_from_phone2 = Phone_cipher2.encrypt(encrypted_response_from_phone1)
countbytes(encrypted_response_from_phone2)

original: b'\x86\x02u\x8dct\x8a\xe8'
byte value : b'\x86\x02u\x8dct\x8a\xe8'
size : 8 bytes  
original: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AUTOCRYPT'
byte value : b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AUTOCRYPT'
size : 64 bytes  
original: b'\x86\x02u\x8dct\x8a\xe8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AUTOCRYPT'
byte value : b'\x86\x02u\x8dct\x8a\xe8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

### Car respose check (challenge solve check)

In [12]:
#AES key는 16의 배수이므로 키의 길이를 32바이트로 만들기 위한 패딩작업 
AES_key1 = AES_key_padding(str(carSharedKey)) # 1st encryption with shared key 
AES_key2 = AES_key_padding(str(carConfirmationKey)) # 2nd encryption with confirmation key 

Car_cipher1 = AESCipher(key=AES_key1) # 2중 암호화를 위해 Shared password를 Key로 암호화 인스턴스1 생성 
Car_cipher2 = AESCipher(key=AES_key2)  # 2중 암호화를 위해 Ke를 Key로 암호화 인스턴스2 생성 

#2중 복호화 
decrypted_response_from_phone1 = Car_cipher2.decrypt(encrypted_response_from_phone2)
decrypted_response_from_phone = Car_cipher1.decrypt(decrypted_response_from_phone1)

In [13]:
decrypted_response_from_phone

b'\x86\x02u\x8dct\x8a\xe8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00AUTOCRYPT'

In [14]:
responsed_nonce_c = decrypted_response_from_phone[0:4]
responsed_timestamp_c = decrypted_response_from_phone[4:8]
responsed_username = decrypted_response_from_phone[8:]
unpad_responsed_username = responsed_username.decode().lstrip("\x00")

In [16]:
nowtime = int(datetime.datetime.timestamp(datetime.datetime.now()))
challenge_time = int(responsed_timestamp_c.hex(),16)
elapsed_minute = (nowtime-challenge_time)/60 

In [17]:
elapsed_minute

0.11666666666666667

In [18]:
# 2분 이내에 challenge 해결해야할 경우 
elapsed_minute < 10 

True

### Save Shared Key (Ke) to safe location


In [28]:
# RPS 시스템은 안전한 저장을 위해 NXP 칩셋 분석 필요 
# safe_file = open("./car_key","w")
# safe_file.write(str(carSecretKey))
# safe_file.close()

# safe_file = open("./phone_key","w")
# safe_file.write(str(phoneSecretKey))
# safe_file.close()

safe_file = open("./car_shared_key","wb")
safe_file.write(carSharedKey.to_bytes((carSharedKey.bit_length() + 7) // 8, 'big'))
safe_file.close()

safe_file = open("./phone_shared_key","wb")
safe_file.write(phoneSharedKey.to_bytes((phoneSharedKey.bit_length() + 7) // 8, 'big'))
safe_file.close()
